In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [23]:
mx.context.num_gpus()

0

In [33]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
df.fillna(0, inplace=True)

In [34]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [35]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [36]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [37]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [38]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [39]:
train_df.shape

(77145, 21)

In [40]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [41]:
train_ds = PandasDataset(
    train_df,
    target = 'TWS_S26',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'WS_S1', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

In [42]:
test_ds = PandasDataset(
    test_df,
    target = 'TWS_S26',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'WS_S1', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [83]:
K = 12
n_hours = 24*3

In [85]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=200,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=64,
    trainer=trainer,
    dropout_rate=0.3,
)

model = estimator.train(train_ds)

 60%|██████████████████████████████                    | 120/200 [11:26<07:18,  5.49s/it, epoch=3/150, avg_epoch_loss=-4.82]WARNING:gluonts.trainer:Batch [122] of Epoch[2] gave NaN loss and it will be ignored


 86%|███████████████████████████████████████████▎      | 173/200 [16:23<02:30,  5.59s/it, epoch=4/150, avg_epoch_loss=-4.95]WARNING:gluonts.trainer:Batch [174] of Epoch[3] gave NaN loss and it will be ignored


 56%|████████████████████████████                      | 112/200 [10:37<08:35,  5.86s/it, epoch=7/150, avg_epoch_loss=-5.06]WARNING:gluonts.trainer:Batch [113] of Epoch[6] gave NaN loss and it will be ignored


 98%|█████████████████████████████████████████████████ | 196/200 [18:48<00:22,  5.73s/it, epoch=9/150, avg_epoch_loss=-5.12]WARNING:gluonts.trainer:Batch [197] of Epoch[8] gave NaN loss and it will be ignored


 32%|███████████████▊                                  | 63/200 [05:56<13:04,  5.72s/it, epoch=12/150, avg_epoch_loss=-5.24]WARNING:gluonts.trainer:Batch [64] of Epoch[11] gave NaN loss and it will be ignored


...

 76%|████████████████████████████████████▋           | 153/200 [15:11<04:37,  5.90s/it, epoch=143/150, avg_epoch_loss=-5.23]WARNING:gluonts.trainer:Batch [154] of Epoch[142] gave NaN loss and it will be ignored


  0%|                                                                                               | 0/200 [00:00<?, ?it/s]WARNING:gluonts.trainer:Batch [2] of Epoch[144] gave NaN loss and it will be ignored


 64%|██████████████████████████████▋                 | 128/200 [12:39<07:09,  5.96s/it, epoch=147/150, avg_epoch_loss=-5.23]WARNING:gluonts.trainer:Batch [130] of Epoch[146] gave NaN loss and it will be ignored


 34%|████████████████▋                                | 68/200 [06:44<12:57,  5.89s/it, epoch=150/150, avg_epoch_loss=-5.25]WARNING:gluonts.trainer:Batch [69] of Epoch[149] gave NaN loss and it will be ignored


100%|████████████████████████████████████████████████| 200/200 [19:43<00:00,  5.92s/it, epoch=150/150, avg_epoch_loss=-5.23]


#### predicted y

In [88]:
# forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
# forecasts = list(forecast_it)
# tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [63]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [64]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S26',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'WS_S1', 'PUMP_S26', 'MEAN_RAIN',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006]
(12,)
------
1
[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994]
(24,)
------
2
[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994
 1.76999998 1.12       0.87       0.30500001 0.55000001 1.01499999
 1.52999997 2.31999993 2.58999991 2.6099999  2.78500009 2.07999992]
(36,)
------
3
[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37     

[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994
 1.76999998 1.12       0.87       0.30500001 0.55000001 1.01499999
 1.52999997 2.31999993 2.58999991 2.6099999  2.78500009 2.07999992
 1.22000003 0.69999999 0.23       0.405      1.09000003 1.83500004
 2.41499996 2.93000007 3.06500006 2.8599999  2.10500002 1.61000001
 0.87       0.32499999 0.52999997 1.11000001 1.65499997 2.3599999
 2.72000003 3.02999997 2.875      2.24000001 1.875      1.29499996
 0.28999999 0.405      1.04499996 1.70000005 2.47000003 2.91000009
 3.00999999 2.92499995 2.         1.68499994 1.08500004 0.38
 0.41499999 0.98500001 1.88499999 2.47000003 2.88000011 3.08500004
 2.89499998 2.21000004 1.87       1.29499996 0.47499999 0.19499999
 1.005      1.89999998 2.31999993 2.88000011 3.05999994 2.86999989
 2.2449

[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994
 1.76999998 1.12       0.87       0.30500001 0.55000001 1.01499999
 1.52999997 2.31999993 2.58999991 2.6099999  2.78500009 2.07999992
 1.22000003 0.69999999 0.23       0.405      1.09000003 1.83500004
 2.41499996 2.93000007 3.06500006 2.8599999  2.10500002 1.61000001
 0.87       0.32499999 0.52999997 1.11000001 1.65499997 2.3599999
 2.72000003 3.02999997 2.875      2.24000001 1.875      1.29499996
 0.28999999 0.405      1.04499996 1.70000005 2.47000003 2.91000009
 3.00999999 2.92499995 2.         1.68499994 1.08500004 0.38
 0.41499999 0.98500001 1.88499999 2.47000003 2.88000011 3.08500004
 2.89499998 2.21000004 1.87       1.29499996 0.47499999 0.19499999
 1.005      1.89999998 2.31999993 2.88000011 3.05999994 2.86999989
 2.2449

[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994
 1.76999998 1.12       0.87       0.30500001 0.55000001 1.01499999
 1.52999997 2.31999993 2.58999991 2.6099999  2.78500009 2.07999992
 1.22000003 0.69999999 0.23       0.405      1.09000003 1.83500004
 2.41499996 2.93000007 3.06500006 2.8599999  2.10500002 1.61000001
 0.87       0.32499999 0.52999997 1.11000001 1.65499997 2.3599999
 2.72000003 3.02999997 2.875      2.24000001 1.875      1.29499996
 0.28999999 0.405      1.04499996 1.70000005 2.47000003 2.91000009
 3.00999999 2.92499995 2.         1.68499994 1.08500004 0.38
 0.41499999 0.98500001 1.88499999 2.47000003 2.88000011 3.08500004
 2.89499998 2.21000004 1.87       1.29499996 0.47499999 0.19499999
 1.005      1.89999998 2.31999993 2.88000011 3.05999994 2.86999989
 2.2449

[2.88499999 2.38499999 1.67499995 1.13499999 0.83499998 0.435
 0.55000001 1.005      1.78499997 2.13000011 2.70000005 2.81500006
 2.39499998 1.75       1.37       0.81999999 0.315      0.51499999
 0.94999999 1.66999996 2.20000005 2.5        2.74000001 2.80999994
 1.76999998 1.12       0.87       0.30500001 0.55000001 1.01499999
 1.52999997 2.31999993 2.58999991 2.6099999  2.78500009 2.07999992
 1.22000003 0.69999999 0.23       0.405      1.09000003 1.83500004
 2.41499996 2.93000007 3.06500006 2.8599999  2.10500002 1.61000001
 0.87       0.32499999 0.52999997 1.11000001 1.65499997 2.3599999
 2.72000003 3.02999997 2.875      2.24000001 1.875      1.29499996
 0.28999999 0.405      1.04499996 1.70000005 2.47000003 2.91000009
 3.00999999 2.92499995 2.         1.68499994 1.08500004 0.38
 0.41499999 0.98500001 1.88499999 2.47000003 2.88000011 3.08500004
 2.89499998 2.21000004 1.87       1.29499996 0.47499999 0.19499999
 1.005      1.89999998 2.31999993 2.88000011 3.05999994 2.86999989
 2.2449

...

[2.88499999 2.38499999 1.67499995 ... 1.92999995 1.80999994 1.58000004]
(226152,)
------
18846
[2.88499999 2.38499999 1.67499995 ... 1.92499995 1.80499995 1.04999995]
(226164,)
------
18847
[2.88499999 2.38499999 1.67499995 ... 1.59000003 1.13499999 0.67500001]
(226176,)
------
18848
[2.88499999 2.38499999 1.67499995 ... 1.06500006 0.57499999 0.025     ]
(226188,)
------
18849
[2.88499999 2.38499999 1.67499995 ... 0.67000002 0.1        0.        ]
(226200,)
------
18850
[2.88499999 2.38499999 1.67499995 ... 0.07       0.         0.205     ]
(226212,)
------
18851
[2.88499999 2.38499999 1.67499995 ... 0.         0.145      1.23000002]
(226224,)
------
18852
[2.88499999 2.38499999 1.67499995 ... 0.27000001 1.29999995 1.80499995]
(226236,)
------
18853
[2.88499999 2.38499999 1.67499995 ... 1.24000001 1.70500004 2.05999994]
(226248,)
------
18854
[2.88499999 2.38499999 1.67499995 ... 1.53999996 2.17000008 2.19499993]
(226260,)
------
18855
[2.88499999 2.38499999 1.67499995 ... 1.87       2

[2.88499999 2.38499999 1.67499995 ... 2.         1.90999997 1.61000001]
(227196,)
------
18933
[2.88499999 2.38499999 1.67499995 ... 1.72500002 1.495      1.34000003]
(227208,)
------
18934
[2.88499999 2.38499999 1.67499995 ... 1.495      1.43499994 1.04499996]
(227220,)
------
18935
[2.88499999 2.38499999 1.67499995 ... 1.33000004 1.01999998 0.875     ]
(227232,)
------
18936
[2.88499999 2.38499999 1.67499995 ... 1.005      0.935      0.995     ]
(227244,)
------
18937
[2.88499999 2.38499999 1.67499995 ... 0.79000002 0.86000001 0.995     ]
(227256,)
------
18938
[2.88499999 2.38499999 1.67499995 ... 0.875      1.08500004 1.35500002]
(227268,)
------
18939
[2.88499999 2.38499999 1.67499995 ... 1.20500004 1.42999995 1.67499995]
(227280,)
------
18940
[2.88499999 2.38499999 1.67499995 ... 1.48000002 1.77499998 1.96500003]
(227292,)
------
18941
[2.88499999 2.38499999 1.67499995 ... 1.69000006 1.90499997 2.07999992]
(227304,)
------
18942
[2.88499999 2.38499999 1.67499995 ... 1.92999995 2

[2.88499999 2.38499999 1.67499995 ... 1.94000006 1.745      1.58500004]
(228240,)
------
19020
[2.88499999 2.38499999 1.67499995 ... 1.875      1.45000005 1.34500003]
(228252,)
------
19021
[2.88499999 2.38499999 1.67499995 ... 1.59000003 1.36500001 1.28999996]
(228264,)
------
19022
[2.88499999 2.38499999 1.67499995 ... 1.52999997 1.25       1.10500002]
(228276,)
------
19023
[2.88499999 2.38499999 1.67499995 ... 1.16999996 1.15999997 1.005     ]
(228288,)
------
19024
[2.88499999 2.38499999 1.67499995 ... 0.98500001 1.11500001 1.24000001]
(228300,)
------
19025
[2.88499999 2.38499999 1.67499995 ... 1.09500003 1.27999997 1.55999994]
(228312,)
------
19026
[2.88499999 2.38499999 1.67499995 ... 1.19500005 1.35000002 1.58500004]
(228324,)
------
19027
[2.88499999 2.38499999 1.67499995 ... 1.33000004 1.52499998 1.67499995]
(228336,)
------
19028
[2.88499999 2.38499999 1.67499995 ... 1.61500001 1.79999995 1.90499997]
(228348,)
------
19029
[2.88499999 2.38499999 1.67499995 ... 1.61500001 1

[2.88499999 2.38499999 1.67499995 ... 1.33500004 1.34500003 1.07500005]
(229284,)
------
19107
[2.88499999 2.38499999 1.67499995 ... 1.29999995 1.10500002 0.76499999]
(229296,)
------
19108
[2.88499999 2.38499999 1.67499995 ... 1.02499998 0.73500001 0.105     ]
(229308,)
------
19109
[2.88499999 2.38499999 1.67499995 ... 0.83999997 0.13500001 0.005     ]
(229320,)
------
19110
[2.88499999 2.38499999 1.67499995 ... 0.075      0.01       0.025     ]
(229332,)
------
19111
[2.88499999 2.38499999 1.67499995 ... 0.01       0.01       0.495     ]
(229344,)
------
19112
[2.88499999 2.38499999 1.67499995 ... 0.01       0.47       1.04499996]
(229356,)
------
19113
[2.88499999 2.38499999 1.67499995 ... 0.53500003 1.16499996 1.44000006]
(229368,)
------
19114
[2.88499999 2.38499999 1.67499995 ... 1.21500003 1.41499996 1.65999997]
(229380,)
------
19115
[2.88499999 2.38499999 1.67499995 ... 1.38       1.79499996 1.66999996]
(229392,)
------
19116
[2.88499999 2.38499999 1.67499995 ... 1.60500002 1

[2.88499999 2.38499999 1.67499995 ... 1.88499999 1.60500002 1.22500002]
(230328,)
------
19194
[2.88499999 2.38499999 1.67499995 ... 1.56500006 1.255      0.95499998]
(230340,)
------
19195
[2.88499999 2.38499999 1.67499995 ... 1.19000006 0.88       0.30500001]
(230352,)
------
19196
[2.88499999 2.38499999 1.67499995 ... 0.88499999 0.245      0.125     ]
(230364,)
------
19197
[2.88499999 2.38499999 1.67499995 ... 0.26499999 0.115      0.25999999]
(230376,)
------
19198
[2.88499999 2.38499999 1.67499995 ... 0.17       0.25999999 0.94      ]
(230388,)
------
19199
[2.88499999 2.38499999 1.67499995 ... 0.205      0.80500001 1.30999994]
(230400,)
------
19200
[2.88499999 2.38499999 1.67499995 ... 0.89499998 1.24000001 1.44000006]
(230412,)
------
19201
[2.88499999 2.38499999 1.67499995 ... 1.38999999 1.505      1.45500004]
(230424,)
------
19202
[2.88499999 2.38499999 1.67499995 ... 1.65499997 1.59500003 1.86000001]
(230436,)
------
19203
[2.88499999 2.38499999 1.67499995 ... 1.92499995 1

In [65]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19204, 12)

In [66]:
test_ds_predict_q50_supervised[0:5]

array([[2.88499999, 2.38499999, 1.67499995, 1.13499999, 0.83499998,
        0.435     , 0.55000001, 1.005     , 1.78499997, 2.13000011,
        2.70000005, 2.81500006],
       [2.39499998, 1.75      , 1.37      , 0.81999999, 0.315     ,
        0.51499999, 0.94999999, 1.66999996, 2.20000005, 2.5       ,
        2.74000001, 2.80999994],
       [1.76999998, 1.12      , 0.87      , 0.30500001, 0.55000001,
        1.01499999, 1.52999997, 2.31999993, 2.58999991, 2.6099999 ,
        2.78500009, 2.07999992],
       [1.22000003, 0.69999999, 0.23      , 0.405     , 1.09000003,
        1.83500004, 2.41499996, 2.93000007, 3.06500006, 2.8599999 ,
        2.10500002, 1.61000001],
       [0.87      , 0.32499999, 0.52999997, 1.11000001, 1.65499997,
        2.3599999 , 2.72000003, 3.02999997, 2.875     , 2.24000001,
        1.875     , 1.29499996]])

#### actual y

In [67]:
from helper import series_to_supervised, stage_series_to_supervised

In [68]:
test_df_supervised = series_to_supervised(test_df[['TWS_S26']], n_hours, K)

In [69]:
test_df_supervised.iloc[:, -K:].shape

(19204, 12)

In [70]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [71]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),var1(t+10),var1(t+11)
77217,2.57,1.97,1.57,1.06,0.59,0.24,0.55,1.02,1.64,2.13,2.56,2.77
77218,1.97,1.57,1.06,0.59,0.24,0.55,1.02,1.64,2.13,2.56,2.77,2.63
77219,1.57,1.06,0.59,0.24,0.55,1.02,1.64,2.13,2.56,2.77,2.63,2.11
77220,1.06,0.59,0.24,0.55,1.02,1.64,2.13,2.56,2.77,2.63,2.11,1.56
77221,0.59,0.24,0.55,1.02,1.64,2.13,2.56,2.77,2.63,2.11,1.56,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...
96416,1.85,2.23,2.39,2.28,1.79,1.28,0.71,0.11,0.07,0.21,0.75,1.29
96417,2.23,2.39,2.28,1.79,1.28,0.71,0.11,0.07,0.21,0.75,1.29,1.66
96418,2.39,2.28,1.79,1.28,0.71,0.11,0.07,0.21,0.75,1.29,1.66,2.04
96419,2.28,1.79,1.28,0.71,0.11,0.07,0.21,0.75,1.29,1.66,2.04,2.18


#### performance on test set

In [72]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.332
RMSE = 0.431
